In [ ]:
# Import libraries
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import joblib 
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
# Load json file
df = pd.read_json('News_Category_Dataset_v3.json', lines=True)

# Keep only relevant columns
df = df[['headline', 'category']]

# Factorize category_id
df['category_id'], categories = pd.factorize(df['category'])
df['category_id'] = df['category_id'] + 1

# Dropping all rows containing NaN
df = df.dropna()

In [ ]:
# Define a function to remove stop words and stem words
def preprocess(text):
    # Convert text to lowercase
    text = text.lower()
    # Tokenize text into words
    words = nltk.word_tokenize(text)
    # Remove stop words
    stop_words = nltk.corpus.stopwords.words('english')
    words = [w for w in words if w not in stop_words]
    # Stem words using Porter stemmer
    stemmer = nltk.stem.PorterStemmer()
    words = [stemmer.stem(w) for w in words]
    # Return the processed text as a string
    return ' '.join(words)

# Apply the preprocess function to the headline column
df['headline'] = df['headline'].apply(preprocess)

In [ ]:
# Oversample the minority class to address class imbalance
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(df[['headline']], df['category_id'])

# Get the category we need for testing
X = X_resampled['headline']
y = y_resampled

# Create test/train split
X_train, X_test, y_train, y_test = \
train_test_split(X, y, test_size = 0.30, random_state = 90)
print(X_train.shape)
print(X_test.shape)

In [ ]:
# Initiate the model
lr = Pipeline([('cv', TfidfVectorizer()),
               ('clf', LogisticRegression(max_iter = 10000)),
              ])

# Train the logistic regression model on the training set
lr.fit(X_train,y_train)

In [ ]:
# Make predictions on the test set
y_pred = lr.predict(X_test)

# Calculate the accuracy of the model
print(f"Accuracy is: {accuracy_score(y_pred,y_test)}")

In [ ]:
# Save the model
joblib.dump(lr, 'lr_model.joblib')

In [ ]:
# Test model on random healdines
news = ["Biden to Sign Executive Order That Aims to Make Child Care Cheaper",
       "Google Stock Loses $57 Billion Amid Microsoft's AI 'Lead'—And \
       Reports It Could Be Replaced By Bing On Some Smartphones",
       "Poland suspends food imports from Ukraine to assist its farmers",
       "Can AI Solve The Air Traffic Control Problem? Let's Find Out",
       "Woman From Odisha Runs 42.5 KM In UK Marathon Wearing A Saree",
       "Hillary Clinton: Trump cannot win the election - but Biden will",
       "Jennifer Aniston and Adam Sandler starrer movie 'Murder Mystery 2' \
       got released on March 24, this year"]

predicted = lr.predict(news)
     
for i in range(len(news)): 
    print(f"{categories[predicted[i]-1]} : {news[i]}")